# PSからPLを制御するためのプログラム

## import

In [5]:
import numpy as np
from time import sleep
from pynq import PL, Overlay, MMIO, allocate

## PL制御クラス

In [88]:
class Kmj_Gen:
  def __init__(self, bitfile, N=10):
    PL.reset()

    ol = Overlay(bitfile)

    dma = ol.axi_dma_0
    self.dma_send = dma.sendchannel
    self.dma_recv = dma.recvchannel

    self.top_mmio = MMIO(ol.ip_dict['top_0']['phys_addr'], ol.ip_dict['top_0']['addr_range'])
    # slv_reg0[2:0] : {set, run, rst_n}
    self.top_mmio.write(0*4, 0b000)
    
    self.input_buffer  = allocate(shape=(N,), dtype=np.uint8)
    self.output_buffer = allocate(shape=(N,), dtype=np.uint8)
  
  def send_data(self, data):
    for i in range(len(self.input_buffer)):
      self.input_buffer[i] = data[i]
    
    self.dma_send.transfer(self.input_buffer)
    while not self.dma_send.idle:
      sleep(0.001)
  
  def recv_data(self):
    self.dma_recv.transfer(self.output_buffer)
    while not self.dma_recv.idle:
      sleep(0.001)
    return self.output_buffer
  
  def run_mode(self, mode=0):
    # slv_reg1[1:0] : mode
    self.top_mmio.write(1*4, mode)

    # slv_reg0[2:0] : {set, run, rst_n}
    self.top_mmio.write(0*4, 0b101)

    # slv_reg0[2:0] : {set, run, rst_n}
    self.top_mmio.write(0*4, 0b011)

    # slv_reg2[0] : finish
    while self.top_mmio.read(2*4) != 1:
      sleep(0.001)


## 関数

In [89]:
# 使用できる文字のリストを読み込み
char_list = []
with open('char_list.txt' , 'r', encoding='utf-8') as file:
  for line in file:
    char_list.append(line.replace('\n', ''))


# 文字リストの番号を顔文字中の番号を対応付け
def convert_str_int(kmj_data, N=10):
  kmj_index = []
  for kmj in kmj_data:
    kmj = list(kmj)
    kmj += ['<PAD>' for _ in range(N - len(kmj))]
    temp = []
    for c in kmj:
      try:
        temp.append(char_list.index(c))
      except:
        temp.append(char_list.index('<UNK>'))
    kmj_index.append(temp)
  
  return kmj_index


# 整数列を文字列に変換
def convert_int_str(x):
  x = np.array(char_list)[x]
  x = [c for c in x if c not in ['<PAD>', '<UNK>']]

  return ''.join(x)

## 動作検証

In [154]:
# PL制御クラスのインスタンス
pl = Kmj_Gen('kmj_gen_v2_2_100MHz.bit')

In [155]:
# 顔文字を整数に変換
# kmj_data = ['(>_<)']
kmj_data = ['ヾ(*　∀́　*)ノ']
kmj_index = convert_str_int(kmj_data)

In [156]:
# データを送信
pl.send_data(kmj_index[0])

In [157]:
# モードを決めて実行
# FORWARD:0, GEN_SIMI:2, GEN_NEW:3
pl.run_mode(mode=2)

In [158]:
# データを受信
output = pl.recv_data()
print(kmj_data[0])
print(convert_int_str(output))

ヾ(*　∀́　*)ノ
ヾ(*　ώ　*)ノ


In [159]:
print(kmj_data[0])
for _ in range(100):
    # データを送信 (GEN_NEWのときはコメントアウト)
    pl.send_data(kmj_index[0])
    
    # モードを決めて実行
    # FORWARD:0, GEN_SIMI:2, GEN_NEW:3
    pl.run_mode(mode=2)
    
    # データを受信
    output = pl.recv_data()
    print(convert_int_str(output))

ヾ(*　∀́　*)ノ
ヾ(*　Д́　*)ノ
ヾ(`　ώ　*)ノ
ヾ(*　ώ　*)ノ
!(*　ώ　*)o
ヾ(*　∀́　*)ノ
\(*　Д́≧*)ノ
ヾ(*　∀́　*)ヽ
~(*　ώ　*)ノ
ヾ(*　Д́　`)ヽ
ヾ(*≧Д́　;ヾヽ
ヾ(*　ώ　*)ノ
\(*　ώ　*)ノ
ヾ(*　ώ　;)ノ
ヾ(*　Д́　*)ヽ
ヾ(*　Д́　*)ノ
((*　Д́　*)ノ
ヾ(*　Д́　*)ノ
ヾ(*　ώ　*)ヽ
ヾ(*　ώ　*)ノ
\(*　Д́　*)ノ
ヾ(*　ώ　;)ノ
ヾ(*　Д́　;)ノ
ヾ(*≧Д́　;)ノ
ヾ(`　ώ　*)ノ
\(*　Д́　*)ノ
ヾ(*　ώ　;)ノ
\(*　Д́　;)ノ
((`　∀́　`)ノ
\(*　Д́　*)ノ
ヾ(*　ώ　*)ノ
ヾ(*　ώ　*)ノ
ヾ(*　Д́　*)ノ
\(*　Д'　;)ノ
\o`　Д́　;)ノ
!(*　ώ　*)ノ
ヾ(*≧Д́　*)ノ
ゞ(*　ώ　*)ノ
\(*≧Д́　;)ノ
ヾ(*　Д́　*)ノ
ヾ(*　∀́　;)ヽ
ヾ(*　Д́　*)ノ
ヾ(*　Д́　`)ヽ
\(́　Д́　*)ノ
ヾ(*　Д́　*)ヽ
\(*　Д́　*)ノ
ヾ(*　Д́　;)ノ
ゞ(*　Д́　*)ノ
ヾ(*　Д́　*)ノ
ヾ(*　Д́　*)ヽ
ヾ(*　Д́　*)ヽ
((*≧Д́　*)ノ
ヾ(*　Д́　*)ノ
ヾ(*　ώ　*)ノ
ヾ(*　ώ　;)ノ
ヾ(*　Д́　*)ノ
\(≧　ώ　;)ノ
ヾ(*　Д́　;)ノ
ヾ(*≧ώ　*)ノ
ヾ(*　Д́　*)ノ
ヾ(*　Д́　;)ヽ
ヾ(*　ώ　*)ノ
ヾ(*　Д́　*)ヽ
ヾ(*　Д́　;)ノ
\(*　Д́　`)ノ
\(*　Д́　;)ノ
ヾ(*　Д́　;)ノ
ヾ(*　Д́　*)ノ
ヾ(*　д́　;)ノ
~(*　ώ　*)ノ
ヾ(*　Д́　;)ノ
ヽ(*　Д́≧*)ノ
♪(*　ώ　`)ノ
\(*　Д́　;)ノ
ヾ(*　ώ　*)ノ
ヾ(*　Д́　;)ノ
ヾ(́　ώ　*)ノ
ヾ(*　ώ　;)ノ
ヾ(*　Д́　*)ヽ
ヾ(*　Д́　;)ノ
\(*　Д́　*)ヽ
ヾ(*　ώ　*)ノ
ヾ(*　Д́　*)ヽ
ヾ(*　Д́　*)ヽ
ヾ(*　Д́　*)ノ
♪(*≧д́　*)ノ
ヾ(*　Д́　*)ノ
ヾ(*　ώ≧`)ノ
ヾ(*　□́≧*)ヽ
ヾ(*　∀́　*)ノ
ヾ(*　∀́　*)ノ